In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data
from matrix_factorization import MF, MF_CVIB, MF_IPS, MF_SNIPS, MF_DR
from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"

In [2]:
if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300


In [3]:
"MF CVIB"
mf_cvib = MF_CVIB(num_user, num_item)
mf_cvib.fit(x_train, y_train, 
    lr=0.01,
    batch_size=128,
    lamb=1e-4,
    alpha=1.0,
    gamma=1e-5,
    tol=1e-5,
    verbose=False)

test_pred = mf_cvib.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_cvib, x_test, y_test)

print("***"*5 + "[MF-CVIB]" + "***"*5)
print("[MF-CVIB] test mse:", mse_func(y_test, test_pred))
print("[MF-CVIB] test auc:", auc_mf)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-CVIB]" + "***"*5)

[MF-CVIB] epoch:52, xent:30.549777269363403
***************[MF-CVIB]***************
[MF-CVIB] test mse: 0.295838087273381
[MF-CVIB] test auc: 0.719465375100819
[MF] ndcg@5:0.652269, ndcg@10:0.714792
Num User: 290
Gini index: 0.31456930008475814
Global utility: 0.5331034482758621
***************[MF-CVIB]***************


In [4]:
"MF naive"
mf = MF(num_user, num_item)
mf.fit(x_train, y_train, 
    lr=0.01,
    batch_size=128,
    lamb=1e-4,
    tol=1e-5,
    verbose=False)
test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf, x_test, y_test)

print("***"*5 + "[MF]" + "***"*5)
print("[MF] test mse:", mse_func(y_test, test_pred))
print("[MF] test auc:", auc_mf)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF]" + "***"*5)

[MF] epoch:97, xent:17.09231725335121
***************[MF]***************
[MF] test mse: 0.24508869837609376
[MF] test auc: 0.7027930246783265
[MF] ndcg@5:0.592044, ndcg@10:0.670445
Num User: 290
Gini index: 0.34384441993189874
Global utility: 0.49586206896551727
***************[MF]***************


In [5]:
"MF IPS"
mf_ips = MF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_ips.fit(x_train, y_train,  y_ips=y_ips,
    lr=0.01,
    batch_size=128,
    lamb=1e-4,
    tol=1e-5,
    verbose=False)
test_pred = mf_ips.predict(x_test)
mse_mfips = mse_func(y_test, test_pred)
auc_mfips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_ips, x_test, y_test)

print("***"*5 + "[MF-IPS]" + "***"*5)
print("[MF-IPS] test mse:", mse_func(y_test, test_pred))
print("[MF-IPS] test auc:", auc_mfips)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-IPS]" + "***"*5)

[MF-IPS] epoch:107, xent:150.0730266571045
***************[MF-IPS]***************
[MF-IPS] test mse: 0.2910913378554706
[MF-IPS] test auc: 0.6839046861213509
[MF] ndcg@5:0.597300, ndcg@10:0.678137
Num User: 290
Gini index: 0.345679969004263
Global utility: 0.4910344827586206
***************[MF-IPS]***************


In [6]:
"MF-SNIPS"
mf_snips = MF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_snips.fit(x_train, y_train,  y_ips=y_ips,
    lr=0.01,
    batch_size=128,
    lamb=1e-4,
    tol=1e-5,
    verbose=False)
test_pred = mf_snips.predict(x_test)
mse_mfsnips = mse_func(y_test, test_pred)
auc_mfsnips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_snips, x_test, y_test)

print("***"*5 + "[MF-SNIPS]" + "***"*5)
print("[MF-SNIPS] test mse:", mse_mfsnips)
print("[MF-SNIPS] test auc:", auc_mfsnips)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-SNIPS]" + "***"*5)

[MF-SNIPS] epoch:96, xent:16.663922056555748
***************[MF-SNIPS]***************
[MF-SNIPS] test mse: 0.24523120279593955
[MF-SNIPS] test auc: 0.6882768282941232
[MF] ndcg@5:0.609800, ndcg@10:0.680055
Num User: 290
Gini index: 0.32996659923789917
Global utility: 0.5055172413793103
***************[MF-SNIPS]***************


In [7]:
"MF DR"
mf_dr = MF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_dr.fit(x_train, y_train,  y_ips=y_ips,
    lr=0.05,
    batch_size=128,
    lamb=1e-4,
    tol=1e-5,
    verbose=False)
test_pred = mf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr, x_test, y_test)

print("***"*5 + "[MF-DR]" + "***"*5)
print("[MF-DR] test mse:", mse_mfdr)
print("[MF-DR] test auc:", auc_mfdr)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-DR]" + "***"*5)

[MF-DR] epoch:53, xent:24142.003189086914
***************[MF-DR]***************
[MF-DR] test mse: 0.2873087573917237
[MF-DR] test auc: 0.6810557518449007
[MF] ndcg@5:0.605651, ndcg@10:0.677353
Num User: 290
Gini index: 0.33659744713278805
Global utility: 0.49931034482758613
***************[MF-DR]***************
